<img src="img/docker-logo.png">
https://www.docker.com/

# Docker

Docker is the most popular application containerization system. For technical reasons, we do not have the opportunity to run and demonstrate Docker capabilities inside Coursera Labs, since they themselves use Docker for their work.

Therefore, to demonstrate the capabilities of docker, we will use the service **play-with-docker.com**, which is sponsored by the developers themselves.
This service allows you to run a virtual environment for 4 hours with docker tools installed.

[Open Play With Docker](https://labs.play-with-docker.com/)

To work with this service, you need to register on the official website docker.com.

Below in the laptop there will be commands that can be run inside play with docker.

The first thing to do is create a new instance (virtual machine). For the current tasks, exactly one machine is enough for us. In the next lab we will analyze how to manage a cluster of machines at once.

# Dockerfile

Let's create a Dockerfile. We will write development documentation in it.

```bash
touch Dockerfile
```

After creating the file, you can click `Editor` and add the file content through the site's graphical interface.
Let's add there the simplest dockerfile that was in the lecture.

```dockerfile
FROM ubuntu:16.04

ENTRYPOINT ["/bin/bash", "-c", "echo hello"]

```

Let's build this container and name it `my-first-container`

```bash
docker build -t my-first-container:latest .
```

Great, the container is ready!

Let's run it

```bash
docker run my-first-container:latest
```

If everything was done correctly, a cheerful hello should be displayed on the screen.


# Containerizing the Python program

Let's try to put together something more complex - for this we need Python.
By default, it is not in the image. To make it appear in our container, you can either install it yourself or use the base image, which already has python.


1 - Install it yourself
```dockerfile
FROM ubuntu:16.04

RUN apt-get -y update && apt-get install python3 -y

ENTRYPOINT ["python3", "-c", "print('hello from python')"]
```

2 - We use a ready-made image
```dockerfile
FROM python:3.7

ENTRYPOINT ["python3", "-c", "print('hello from python')"]
```

We save any option in the Dockerfile, collect and check the performance of our container.

```bash
docker build -t my-python-hello:latest .
docker run my-python-hello:latest
```

# We pack the external script into a container

It's time to add a more complex program to our container.

Let's create application.py, add code to it that would display the current directory and its contents. We will then add this file to our container and run it.


*application.py*
```python
import os

print("Right now I am here - {}".format(os.getcwd()))

content = os.listdir('.')
print("There are {} elements in this directory".format(len(content)))
for element in content:
    print(element)
```


Before assembling, check that the file itself is successfully launched.

```bash
python3 application.py
```


Now let's collect

*Dockerfile*
```dockerfile
FROM python:3.7

COPY application.py application.py

ENTRYPOINT ["python3", "application.py"]
```

and run inside the container


```bash
docker build -t python-ls:latest .
docker run python-ls:latest
```

It is important to note that the output of the program is different! This is because the application in the container is isolated from the main operating system and internally has its own file system.

# Forwarding command line parameters

For the program inside the container to be useful, you need to learn how to communicate with it from the outside.

The most basic way is parameter forwarding. All parameters that will be passed after the container name in the `docker run` command will be passed inside the container as normal parameters.

Let's create a python program that will receive a number and print the word Hello the specified number of times.

*application.py*
```python
import sys

N = int(sys.argv[1])
for i in range(N):
    print("Hello")
   
```

Now let's collect and try to run with parameters

```bash
docker build -t python-hello:latest .
docker run python-hello:latest 3
docker run python-hello:latest 10
```

# Mounting a directory

The next way to interact with the container is to mount the directory. The mounted directory becomes shared between the container and the host operating system. This allows you to transfer some files to the docker for processing, as well as receive some file results from the docker.

Modify our previous program to show the state of the `/sync-folder` directory into which we will mount various directories externally.


*application.py*
```python
import os
import sys

target_dir = sys.argv[1]

print("Observe directory - {}".format(target_dir))

content = os.listdir(target_dir)
print("There are {} elements in this directory".format(len(content)))
for element in content:
    print(element)
```

Now let's build and try to run with a mounted directory. To mount, you need to specify the `-v` flag in which you specify, separated by a colon, which directories we want to synchronize. The first directory is from the main OS, the second is inside the container.

```bash
docker build -t python-ls:latest .

docker run -v /root:/sync-folder python-ls:latest /sync-folder
docker run -v /:/sync-folder python-ls:latest /sync-folder
```

If we do not specify the mount options, then such a directory will not appear inside the container and, accordingly, our program will simply crash without finding it.

```bash
docker run python-ls:latest
```

# Port forwarding

The most common way to communicate with a container is port forwarding. For this, two ports are specified - one for the main OS, the second for the container. All connections to the external port will be forwarded to the inside of the container. For forwarding, you need to use the `-p` switch - the format is exactly the same as for` -v`.

To demonstrate this functionality, we will use the http.server module built into python, which brings up the file server in the directory from which it was launched. First, let's just check how it works.


Let's run

```bash
python3 -m http.server --bind 0.0.0.0 8000
```

This should start the file server in the current directory.

To check that it works, click on `OPEN PORT` and specify 8000 there.

In the browser, we should see the contents of the current directory.

Let's try now to pack it inside the container.


*Dockerfile*
```dockerfile
FROM python:3.7

WORKDIR /sync-folder

ENTRYPOINT ["python3", "-m", "http.server", "--bind", "0.0.0.0", "8080"]
```

We collect and check

```bash
docker build -t python-server:latest .
docker run -v /root:/sync-folder -p 9090:8080 python-server:latest
```

Open port 9090 with `OPEN PORT` and check what really works

A diagram of how it works:
<img src="img/python-server-schema.png">

Now this container can be used even on machines that don't have Python installed! All dependencies are already inside the container and only Docker is required to run.